## 라이브러리 설치

In [ ]:
!pip install xlsxwriter

In [ ]:
pip install pmdarima


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import itertools
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import statsmodels.api as sm

from datetime import datetime, timedelta

from dateutil.relativedelta import relativedelta
from warnings import filterwarnings
filterwarnings('ignore')
import os
import itertools
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import statsmodels.api as sm
from datetime import datetime
from dateutil.relativedelta import relativedelta
from warnings import filterwarnings
filterwarnings('ignore')
import os
from sklearn.preprocessing import MinMaxScaler

## 데이터 로드 및 데이터 전처리

In [ ]:
RRP.head(30)

,Requested_Date,Requested_Quantity,PRODNAME
0,2018-11-08,2080,AE
1,2018-11-08,31200,CSA5000
2,2018-11-08,15600,PEMA-580FX
3,2018-11-08,157040,PEMA-HR1000
4,2018-11-08,35360,PEMA-HR1500
5,2018-11-08,46800,PEMA-PR1000
6,2018-11-08,18720,PEMA-SR2000
7,2018-11-08,9360,PEMA-SR3000F
8,2018-11-08,0,CSA4000
9,2018-11-08,0,PEMA-500FR


In [ ]:
RRP = pd.read_excel('/content/drive/MyDrive/MyDrive/new_ABC.xlsx')

In [ ]:
co_df=RRP.copy()

In [ ]:
co_df.set_index('Requested_Date',inplace=True)

In [ ]:
scaler = MinMaxScaler()
co_df[['Requested_Quantity']] = scaler.fit_transform(co_df[['Requested_Quantity']])

## train, test 분할

In [ ]:
std_date = datetime.strptime("2021-04-11", "%Y-%m-%d")

testdate_start = std_date - relativedelta(months=6)
traindate_end = testdate_start - relativedelta(days=1)
traindate_start = (traindate_end - relativedelta(years=1,months=11,days=2))

## 모델학습 및 파라미터 설정 및 검증 결과 출력

In [ ]:
predicted_values_dic = {}
pdq_li = []
r2_li = []
aic_li = []
period = []
r2_remove0_li = []

for i in range(1):
  print('train  :',traindate_start.date(),'~',traindate_end.date())
  print('test   :',testdate_start.date(),'~',std_date.date())
  r2_li1 = []
  r2_remove0_li1 = []
  aic_li1 = []
  pdq_li1 = []
  period.append(testdate_start.strftime('%y%m%d')+'~'+std_date.strftime('%y%m%d'))

  for prod in co_df['PRODNAME'].unique():
    print(f'<<<<<<<<<<<<<<<  {prod}({testdate_start.date()} ~ {std_date.date()})  >>>>>>>>>>>>>>>')
    series = co_df.query(f'PRODNAME=="{prod}"')
    series.drop('PRODNAME',axis=1,inplace=True)
    series = series.resample('W').sum()

    # 데이터 분리

    train = series['Requested_Quantity'][traindate_start:traindate_end]
    test = series['Requested_Quantity'][testdate_start:std_date]


    # Grid_search

    p = d = q = range(0,2)
    pdq = list(itertools.product(p,d,q))
    seasonal_pdq = [(x[0],x[1],x[2],12) for x in list(itertools.product(p,d,q))]
    best_aic = np.inf
    best_pdq = None
    best_seasonal_pdq = None
    tmp_model = None
    best_mdl = None

    for param in pdq:
      for param_seasonal in seasonal_pdq:
        try:
          tmp_mdl = sm.tsa.statespace.SARIMAX(train, exog=None, order=param,
                                              seasonal_order = param_seasonal,
                                              enforce_stationarity=True,
                                              enforce_invertibility=True)
          res = tmp_mdl.fit()
          if res.aic < best_aic:
            best_aic = res.aic
            best_pdq = param
            best_seasonal_pdq = param_seasonal
            best_mdl = tmp_mdl
        except:
          continue
    print("Best SARIMAX{}x{}12 model - AIC : {}".format(best_pdq, best_seasonal_pdq, best_aic))

    # 학습

    res = sm.tsa.statespace.SARIMAX(endog=train, order=best_pdq,
                                seasonal_order = best_seasonal_pdq,
                                enforce_stationarity=True,
                                enforce_invertibility=True).fit()

    aic_li1.append(res.aic)
    pdq_li1.append((best_pdq, best_seasonal_pdq, best_aic))
    display(res.summary())

    try:
      res.plot_diagnostics(figsize=(16,10))
    except:
      pass

    # 시각화

    pred = res.get_prediction(start=test.index.min(),
                            end = test.index.max(),
                            dynamic=True)

    pred_ci = pred.conf_int()

    plt.figure(figsize=(14,7))
    ax = series[traindate_start:std_date].plot(label = 'Observed',color='#006699');
    pred.predicted_mean.plot(ax=ax, label='Prediction', alpha=.7, color='#ff0066');

    ax.fill_between(pred_ci.index,
                    pred_ci.iloc[:,0],
                    pred_ci.iloc[:,1],color='#ff0066', alpha=.25);

    ax.fill_betweenx(ax.get_ylim(), test.index.min(),
                    test.index.max(), alpha=.15, zorder=-1,
                    color='gray')
    ax.set_xlabel('Time (week)')
    ax.set_ylabel(f'weekly SOLD_QUANT({prod})')
    ax.set_title('Test data vs prediction By SARIMA')
    plt.legend(loc='upper left')
    plt.show()

    # 검증(R2_score)

    from sklearn.metrics import r2_score
    predicted_value = pred.predicted_mean
    r2 = r2_score(test,predicted_value)
    r2_li1.append(r2)
    print('R2_score :',r2)
    print()
    print()

    # 모델, 결과값 저장

    res.save(f'/content/drive/MyDrive/MyDrive/pred/{prod}{i}.pkl')
    try:
      df = pd.concat([predicted_value,test],axis=1)
      df.to_excel(f'/content/drive/MyDrive/MyDrive/pred/{prod}{i}.xlsx')
      predicted_values_dic[f'{prod}{i+1}'] = df
      df1 = df[df['Requested_Quantity']!=0]
      r2_remove0_li1.append(r2_score(df1['Requested_Quantity'],df1[0]))
    except:
      predicted_values_dic[f'{prod}{i}'] = predicted_value

  print('='*80)
  std_date = testdate_start - relativedelta(weeks=1)
  testdate_start = std_date - relativedelta(months=6)
  traindate_end = testdate_start - relativedelta(days=1)
  traindate_start = (traindate_end - relativedelta(years=2, months=3))
  r2_li.append(r2_li1)
  aic_li.append(aic_li1)
  pdq_li.append(pdq_li1)
  r2_remove0_li.append(r2_remove0_li1)




# # 결과 저장

# r2 = pd.DataFrame(r2_li)
# r2.columns = co_df['PRODNAME'].unique()
# r2 = r2.T
# r2.columns = period

# aic = pd.DataFrame(aic_li)
# aic.columns = co_df['PRODNAME'].unique()
# aic = aic.T
# aic.columns = period

# pdq = pd.DataFrame(pdq_li)
# pdq.columns = co_df['PRODNAME'].unique()
# pdq = pdq.T
# pdq.columns = period

# r2_remove0 = pd.DataFrame(r2_remove0_li)
# r2_remove0.columns = co_df['PRODNAME'].unique()
# r2_remove0 = r2_remove0.T
# r2_remove0.columns = period

# results = pd.DataFrame()
# path = '/content/drive/MyDrive/sarima_train/국취제'
# file_list = os.listdir(path)
# for file in file_list:
#   df = pd.read_excel(path + '/' + file)
#   prod_name = file[:-5]
#   df['Prod_name'] = prod_name
#   results = pd.concat([results, df])
# results = results[results['Requested_Quantity']!=0] # 마이너스 값 삭제(Requested_Quantity 0일 때 예측나온 값들)
# results.columns=['Time','Predicted_Quantity','Requested_Quantity','Prod_name']
# results = results.set_index('Time').sort_index()

# writer = pd.ExcelWriter(f'/content/drive/MyDrive/국취제/results.xlsx',engine='xlsxwriter')
# results.to_excel(writer, sheet_name='results')
# r2.to_excel(writer, sheet_name='r2')
# aic.to_excel(writer, sheet_name='aic')
# pdq.to_excel(writer, sheet_name='pdq')
# r2_remove0.to_excel(writer, sheet_name='r2_remove0')

# writer.save()

Output hidden; open in https://colab.research.google.com to view.

## 모델 불러와서 제품별 예측량 뽑기

In [ ]:
import pandas as pd
import pickle

# 리스트에 있는 모든 제품들을 정의합니다.
products = ['AE', 'CSA5000', 'PEMA-580FX', 'PEMA-HR1000', 'PEMA-HR1500',
            'PEMA-PR1000', 'PEMA-SR2000', 'PEMA-SR3000F', 'CSA4000',
            'PEMA-500FR', 'PEMA-CR1000', 'PEMA-CSA5000', 'PEMA-SN400',
            'PEMA-SR2000A', 'PEMA-PCM2000', 'PEMA-PCM2000B', 'PEMA-PR2000',
            'CSA4000(PCA)', 'PR1000', 'PEMA-PCR3000E', 'PEMA-SR5000F',
            'PEMA-PCR3000N', 'PEMA-SP1000', 'SRE-200', 'PEMA-SPR',
            'PEMA-HR1000S', 'SRE-110']
product_dataframes = {}
for product in products:
    # 모델 불러오기
    with open(f'/content/drive/MyDrive/MyDrive/pred/{product}0.pkl', 'rb') as file:
        loaded_model = pickle.load(file)

    forecast_steps = 56
    forecast = loaded_model.get_forecast(steps=forecast_steps)
    forecast_values = forecast.predicted_mean
    df = pd.DataFrame(forecast_values, columns=['predicted_mean'])
    df['week'] = df.index - pd.Timedelta(days=6)
    df = df.set_index('week')

    df.rename(columns={'predicted_mean': '예측중량'}, inplace=True)
    start_date = pd.to_datetime('2021-04-26')
    end_date = pd.to_datetime('2021-07-26')
    filtered_df = df[(df.index >= start_date) & (df.index <= end_date)]

    # 정규화된 데이터에서 최소값과 최대값을 구합니다.
    min_val = RRP['Requested_Quantity'].min()
    max_val = RRP['Requested_Quantity'].max()

    # 정규화된 값을 원본 범위로 스케일링합니다.
    filtered_df['예측중량'] = (filtered_df['예측중량'] * (max_val - min_val)) + min_val
    product_dataframes[product] = filtered_df
    # # CSV 파일로 저장합니다.
    # filtered_df.to_csv(f'/content/drive/MyDrive/국취제/{product}.csv')


In [ ]:
product_dataframes

{'AE':                     예측중량
 week                    
 2021-04-26  11907.333913
 2021-05-03  32401.466866
 2021-05-10  22691.837537
 2021-05-17  14904.428773
 2021-05-24   8048.568329
 2021-05-31  13903.569682
 2021-06-07  23980.122279
 2021-06-14  12413.419201
 2021-06-21  10610.165596
 2021-06-28  28524.109707
 2021-07-05  19268.253712
 2021-07-12  11887.814774
 2021-07-19   6652.798102
 2021-07-26  11766.981395,
 'CSA5000':                      예측중량
 week                     
 2021-04-26  253940.323718
 2021-05-03  238740.302422
 2021-05-10  295768.319196
 2021-05-17  213886.516008
 2021-05-24  259680.745317
 2021-05-31  235555.368595
 2021-06-07  241360.298174
 2021-06-14  154716.109843
 2021-06-21  225966.211813
 2021-06-28  210979.960766
 2021-07-05  248418.629817
 2021-07-12  176489.794210
 2021-07-19  228253.274266
 2021-07-26  212437.234236,
 'PEMA-580FX':                      예측중량
 week                     
 2021-04-26   66302.418828
 2021-05-03  134951.920044
 2021-05-10

In [ ]:
product_dataframes

{'AE':                     예측중량
 week                    
 2021-04-26  11907.333913
 2021-05-03  32401.466866
 2021-05-10  22691.837537
 2021-05-17  14904.428773
 2021-05-24   8048.568329
 2021-05-31  13903.569682
 2021-06-07  23980.122279
 2021-06-14  12413.419201
 2021-06-21  10610.165596
 2021-06-28  28524.109707
 2021-07-05  19268.253712
 2021-07-12  11887.814774
 2021-07-19   6652.798102
 2021-07-26  11766.981395,
 'CSA5000':                      예측중량
 week                     
 2021-04-26  253940.323718
 2021-05-03  238740.302422
 2021-05-10  295768.319196
 2021-05-17  213886.516008
 2021-05-24  259680.745317
 2021-05-31  235555.368595
 2021-06-07  241360.298174
 2021-06-14  154716.109843
 2021-06-21  225966.211813
 2021-06-28  210979.960766
 2021-07-05  248418.629817
 2021-07-12  176489.794210
 2021-07-19  228253.274266
 2021-07-26  212437.234236,
 'PEMA-580FX':                      예측중량
 week                     
 2021-04-26   66302.418828
 2021-05-03  134951.920044
 2021-05-10

In [ ]:
product_dataframes

{'AE':                     예측중량
 week                    
 2021-04-26  11907.333913
 2021-05-03  32401.466866
 2021-05-10  22691.837537
 2021-05-17  14904.428773
 2021-05-24   8048.568329
 2021-05-31  13903.569682
 2021-06-07  23980.122279
 2021-06-14  12413.419201
 2021-06-21  10610.165596
 2021-06-28  28524.109707
 2021-07-05  19268.253712
 2021-07-12  11887.814774
 2021-07-19   6652.798102
 2021-07-26  11766.981395,
 'CSA5000':                      예측중량
 week                     
 2021-04-26  253940.323718
 2021-05-03  238740.302422
 2021-05-10  295768.319196
 2021-05-17  213886.516008
 2021-05-24  259680.745317
 2021-05-31  235555.368595
 2021-06-07  241360.298174
 2021-06-14  154716.109843
 2021-06-21  225966.211813
 2021-06-28  210979.960766
 2021-07-05  248418.629817
 2021-07-12  176489.794210
 2021-07-19  228253.274266
 2021-07-26  212437.234236,
 'PEMA-580FX':                      예측중량
 week                     
 2021-04-26   66302.418828
 2021-05-03  134951.920044
 2021-05-10

In [ ]:
product_dfs = []
for product, product_df in product_dataframes.items():
    product_df['week'] = product_df.index  # 인덱스를 week 컬럼으로 추가
    product_df['prodname'] = product  # prodname 컬럼 추가
    product_dfs.append(product_df)

all_products_df = pd.concat(product_dfs, ignore_index=True)
all_products_df = all_products_df.set_index('week')
all_products_df = all_products_df.sort_index()
auto_df=all_products_df
auto_df = auto_df.reset_index()

In [ ]:
auto_df['온도'] = None
auto_df['습도'] = None
auto_df['sold_quant'] = None
auto_df['건설기성액'] = None
auto_df['국내건설수주액'] = None
new_column_order = ['week', 'prodname', '온도', '습도','예측중량','sold_quant', '건설기성액', '국내건설수주액']
auto_df = auto_df[new_column_order]
auto_df.rename(columns={'예측중량':'order_quant'}, inplace=True)
auto_df['week'] = auto_df['week'].dt.strftime('%Y-%m-%d') # dt형식으로 변환

In [ ]:
pip install lxml

In [ ]:


import requests
from bs4 import BeautifulSoup
import pandas as pd
from lxml import etree
import json
import requests
import xml.etree.ElementTree as ET
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import pprint
from pandas import DataFrame
import datetime
from datetime import datetime, timedelta



current_time = datetime.now()
# 현재시점을 불러옴
formatted__date = current_time.strftime("%Y%m")
# 건설api 형식으로 수정함 (년/월)
yesterday = current_time - timedelta(days=1)
# 어제시점을 불러옵니다 (기상청은 현재시점에 전날까지 업데이트)
w_date = yesterday.strftime("%Y%m%d")
# 기상청 api 형식으로 수정함(년/월/일)



data_dict = {
    '901Y104' : '건설기성액',
    '901Y020' : '국내건설수주액'
}

KEY = 'P6B82DH7S3NJPJF38DPV'
PERIOD = 'M'
START_DATE = '202104'
END_DATE = formatted__date

def get_product(KEY, STAT_CD, PERIOD, START_DATE, END_DATE):
    url = 'http://ecos.bok.or.kr/api/StatisticSearch/{}/xml/kr/1/1000/{}/{}/{}/{}/'.format(KEY, STAT_CD, PERIOD, START_DATE, END_DATE)

    response = requests.get(url).content.decode('utf-8')
    xml_obj = BeautifulSoup(response, 'lxml-xml')
    rows = xml_obj.findAll("row")
    return rows


item_list = [
    'STAT_CODE','STAT_NAME','ITEM_CODE1','ITEM_NAME1','ITEM_CODE2','ITEM_NAME2','ITEM_CODE3','ITEM_NAME3','UNIT_NAME',
    'TIME','DATA_VALUE'
]

result_list = list()

for k in data_dict.keys():
    rows = get_product(KEY , k ,PERIOD , START_DATE , END_DATE)

    for p in range(0, len(rows)):
        info_list = list()

        for i in item_list:
            try:
                individual_info = rows[p].find(i).text
            except:
                individual_info = ""
            info_list.append(individual_info)

        result_list.append(info_list)

result_df = pd.DataFrame(result_list , columns = ['통계표코드', '통계명', '통계 항목 1코드', '통계항목2코드', '통계항목명2', '통계항목3코드', '뭔데그럼', '나머지두개는', '단위', '시점', '값']).drop_duplicates()


gunsal_gisung_df = result_df[(result_df['통계명'] == '8.4.2. 건설기성액') &
                            (result_df['통계항목2코드'] == '총기성액') &
                            (result_df['통계항목3코드'] == '경상')].copy()

gunsal_suzu_df = result_df[(result_df['통계명'] == '8.4.1. 국내건설수주액') &
                          (result_df['통계항목2코드'] == '총수주액')].copy()

def extract_year_month(date_str):
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
    return date_obj.year, date_obj.month

# autodf의 년월 추출
auto_df['year'], auto_df['month'] = zip(*auto_df['week'].map(extract_year_month))

# 년월이 일치하는 경우 값을 할당
for _, row in auto_df.iterrows():
    matching_row = gunsal_suzu_df[gunsal_suzu_df['시점'] == f"{row['year']}{row['month']:02d}"]
    if not matching_row.empty:
        auto_df.loc[_, '국내건설수주액'] = matching_row['값'].values[0]

for _, row in auto_df.iterrows():
    matching_row = gunsal_gisung_df[gunsal_gisung_df['시점'] == f"{row['year']}{row['month']:02d}"]
    if not matching_row.empty:
        auto_df.loc[_, '건설기성액'] = matching_row['값'].values[0]




url2 = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
params = {
    'serviceKey': 'm6B9bWsl85oDiUgVP3Fkfrgdj89/ea59MvlX6XqTmaurYr2EhziN/Sii8RhhNULFgxQWj/DudnIBBN4rr1iWeg==',
    'pageNo': '1~80',
    'numOfRows': '900',
    'dataType': 'XML',
    'dataCd': 'ASOS',
    'dateCd': 'DAY',
    'startDt': '20210401',
    'endDt': w_date,   # 현재날짜에 하루전까지만 업데이트 됨
    'stnIds': '232' #천안232
}


w_response = requests.get(url2, params=params)



# 응답 데이터를 XML로 파싱
root = ET.fromstring(w_response.content)

# 데이터를 저장할 리스트 생성
data = []

# XML에서 데이터 추출하여 리스트에 추가
for item in root.iter('item'):
    TA_AVG = item.find('avgTa').text
    avgRhm = item.find('avgRhm').text
    tm = item.find('tm').text


    # 데이터를 딕셔너리 형태로 저장
    data.append({
        'tm': tm,
        'TA_AVG': TA_AVG,
        'avgRhm': avgRhm,

    })

# 데이터프레임 생성
df = pd.DataFrame(data)

for _, row in df.iterrows():
    matching_row = auto_df[auto_df['week']== row['tm']]
    if not matching_row.empty:
        auto_df.loc[matching_row.index,'온도'] = row['TA_AVG']
        auto_df.loc[matching_row.index,'습도'] = row['avgRhm']



auto_df


,week,prodname,온도,습도,order_quant,sold_quant,건설기성액,국내건설수주액,year,month
0,2021-04-26,AE,15.1,47.5,1.190733e+04,None,11514860,17774868,2021,4
1,2021-04-26,PEMA-SPR,15.1,47.5,-9.658918e-09,None,11514860,17774868,2021,4
2,2021-04-26,PEMA-HR1000,15.1,47.5,2.994239e+05,None,11514860,17774868,2021,4
3,2021-04-26,SRE-200,15.1,47.5,8.212241e-18,None,11514860,17774868,2021,4
4,2021-04-26,PEMA-HR1500,15.1,47.5,1.206527e+05,None,11514860,17774868,2021,4
...,...,...,...,...,...,...,...,...,...,...
373,2021-07-26,CSA5000,28.1,68.8,2.124372e+05,None,10992998,14541827,2021,7
374,2021-07-26,PEMA-500FR,28.1,68.8,4.667127e+04,None,10992998,14541827,2021,7
375,2021-07-26,PEMA-PCM2000,28.1,68.8,1.889885e+04,None,10992998,14541827,2021,7
376,2021-07-26,PEMA-SR3000F,28.1,68.8,5.919998e+04,None,10992998,14541827,2021,7


In [ ]:
# CSV 파일로 저장
auto_df.to_excel('api_자동화.xlsx', index=False)